In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# Custom Kernel Layer using RBF Kernel
class RBFLayer(nn.Module):
    def __init__(self, in_features, out_features, gamma=1.0):
        super(RBFLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.gamma = gamma  # Parameter for the RBF kernel
        self.centers = nn.Parameter(torch.randn(out_features, in_features))  # Learnable centers

    def rbf_kernel(self, x, centers):
        # Compute the RBF kernel: exp(-gamma * ||x - centers||^2)
        distances = torch.cdist(x, centers, p=2)  # Euclidean distance
        return torch.exp(-self.gamma * distances**2)

    def forward(self, x):
        # Apply the RBF kernel to the input
        return self.rbf_kernel(x, self.centers)

# Neural Network with a Kernel Layer
class KernelNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, kernel_dim, gamma=1.0):
        super(KernelNN, self).__init__()
        self.kernel_layer = RBFLayer(input_dim, kernel_dim, gamma)
        self.fc1 = nn.Linear(kernel_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.kernel_layer(x)  # Apply the kernel layer
        x = self.relu(self.fc1(x))  # Fully connected layer with ReLU
        x = self.fc2(x)  # Output layer
        return x

# Example usage
input_dim = 2
hidden_dim = 10
output_dim = 1
kernel_dim = 20  # Number of RBF centers
gamma = 1.0  # RBF kernel parameter

# Create the model
model = KernelNN(input_dim, hidden_dim, output_dim, kernel_dim, gamma)

# Example input
x = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Forward pass
output = model(x)
print("Output of the neural network with a kernel layer:")
print(output)

Output of the neural network with a kernel layer:
tensor([[-0.0554],
        [-0.0419],
        [-0.0419]], grad_fn=<AddmmBackward0>)


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# Custom Kernel Layer
class KernelLayer(nn.Module):
    def __init__(self, in_features, out_features, kernel_fn):
        """
        Args:
            in_features (int): Number of input features.
            out_features (int): Number of output features (number of kernel centers).
            kernel_fn (callable): A function that computes the kernel between two tensors.
        """
        super(KernelLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.kernel_fn = kernel_fn
        self.centers = nn.Parameter(torch.randn(out_features, in_features))  # Learnable centers

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).
        Returns:
            torch.Tensor: Transformed tensor of shape (batch_size, out_features).
        """
        # Compute the kernel between input x and the learnable centers
        return self.kernel_fn(x, self.centers)

# Example Kernel Functions
def rbf_kernel(x, centers, gamma=1.0):
    """
    Radial Basis Function (RBF) kernel.
    Args:
        x (torch.Tensor): Input tensor of shape (batch_size, in_features).
        centers (torch.Tensor): Centers tensor of shape (out_features, in_features).
        gamma (float): Kernel parameter.
    Returns:
        torch.Tensor: Kernel output of shape (batch_size, out_features).
    """
    distances = torch.cdist(x, centers, p=2)  # Euclidean distance
    return torch.exp(-gamma * distances**2)

def polynomial_kernel(x, centers, degree=2, c=1.0):
    """
    Polynomial kernel.
    Args:
        x (torch.Tensor): Input tensor of shape (batch_size, in_features).
        centers (torch.Tensor): Centers tensor of shape (out_features, in_features).
        degree (int): Degree of the polynomial.
        c (float): Constant term.
    Returns:
        torch.Tensor: Kernel output of shape (batch_size, out_features).
    """
    return (torch.matmul(x, centers.T) + c) ** degree

# Neural Network with Custom Kernel Layer
class KernelNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, kernel_dim, kernel_fn):
        """
        Args:
            input_dim (int): Number of input features.
            hidden_dim (int): Number of hidden units in the fully connected layer.
            output_dim (int): Number of output features.
            kernel_dim (int): Number of kernel centers.
            kernel_fn (callable): Kernel function to use in the kernel layer.
        """
        super(KernelNN, self).__init__()
        self.kernel_layer = KernelLayer(input_dim, kernel_dim, kernel_fn)
        self.fc1 = nn.Linear(kernel_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.kernel_layer(x)  # Apply the kernel layer
        x = self.relu(self.fc1(x))  # Fully connected layer with ReLU
        x = self.fc2(x)  # Output layer
        return x

# Example usage
input_dim = 2
hidden_dim = 10
output_dim = 1
kernel_dim = 20  # Number of kernel centers

# Define the kernel function (e.g., RBF or polynomial)
kernel_fn = lambda x, centers: rbf_kernel(x, centers, gamma=1.0)  # RBF kernel
# kernel_fn = lambda x, centers: polynomial_kernel(x, centers, degree=2, c=1.0)  # Polynomial kernel

# Create the model
model = KernelNN(input_dim, hidden_dim, output_dim, kernel_dim, kernel_fn)

# Example input
x = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Forward pass
output = model(x)
print("Output of the neural network with a custom kernel layer:")
print(output)

Output of the neural network with a custom kernel layer:
tensor([[0.0078],
        [0.0009],
        [0.0009]], grad_fn=<AddmmBackward0>)


In [6]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Example training loop
for epoch in range(100):
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, torch.tensor([[1.0], [0.0], [1.0]]))  # Example target
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.660855233669281
Epoch 2, Loss: 0.6326214671134949
Epoch 3, Loss: 0.6048409938812256
Epoch 4, Loss: 0.5775526165962219
Epoch 5, Loss: 0.5508005619049072
Epoch 6, Loss: 0.5246261954307556
Epoch 7, Loss: 0.4990682601928711
Epoch 8, Loss: 0.4743987023830414
Epoch 9, Loss: 0.45087575912475586
Epoch 10, Loss: 0.42792555689811707
Epoch 11, Loss: 0.4056072235107422
Epoch 12, Loss: 0.38398146629333496
Epoch 13, Loss: 0.363110214471817
Epoch 14, Loss: 0.34305694699287415
Epoch 15, Loss: 0.3238866627216339
Epoch 16, Loss: 0.3056658208370209
Epoch 17, Loss: 0.28846195340156555
Epoch 18, Loss: 0.2723430097103119
Epoch 19, Loss: 0.2573758065700531
Epoch 20, Loss: 0.24362467229366302
Epoch 21, Loss: 0.23114943504333496
Epoch 22, Loss: 0.2200024574995041
Epoch 23, Loss: 0.2102261334657669
Epoch 24, Loss: 0.20184926688671112
Epoch 25, Loss: 0.19488345086574554
Epoch 26, Loss: 0.1893189698457718
Epoch 27, Loss: 0.18512074649333954
Epoch 28, Loss: 0.18222419917583466
Epoch 29, Loss: 0.18